In [1]:
from collections import Counter
from nltk.sentiment import util
import numpy as np
import pandas as pd
from ast import literal_eval

In [2]:
def make_unigram_feature_set(documents, min_freq=1, mark_negation=False):
    """
    This function goes through a corpus and retains all candidate unigram features
     making a feature set. Optionally, it can also preprocess the corpus annotating
     with _NEG words that are in the scope of a negation (using NLTK helper functions).
    
    :param documents: all documents, each a list of words
    :param min_freq: minimum frequency of a token for it to be part of the feature set
    :param mark_negation: whether to preprocess the document using NLTK's nltk.sentiment.util.mark_negation
        see the documentation `nltk.sentiment.util.mark_negation?`
    :returns: unigram feature set
    """
    counter = Counter()
    for doc in documents:
        if mark_negation:
            util.mark_negation(doc, shallow=True)
        counter.update(doc)
    features = []
    for f, n in counter.most_common():
        if n >= min_freq:
            features.append(f)
        else:
            break
    return frozenset(features)

In [3]:
def inspect_set(input_set, k=5, neg=False):
    """
    Helper function to inspect a few elements in a set of features
    
    :param input_set: a set of features
    :param k: how many elements to select
    :param neg: return `*_NEG` features only
    :returns: up to k elements 
    """
    selected = set()
    for w in input_set:
        if len(selected) < k:            
            if not neg:
                selected.add(w)
            elif '_NEG' in w:
                selected.add(w)
        else:
            break
    return selected

In [4]:
from collections import defaultdict

def make_feature_map(document, feature_set, 
                     binary=True, 
                     mark_negation=False):
    """
    This function takes a document, possibly pre-processes it by marking words in the scope of negation, 
     and constructs a dict indicating which features in `feature_set` fire. Features may be binary, 
     flagging occurrence, or integer, indicating the number of occurrences.
     If no feature can be extracted, a special feature is fired, namely 'EMPTY()'.
     
    :param document: a list of words
    :param feature_set: set of features we are looking for
    :param binary: whether we are indicating presence or counting features in feature_set
    :param mark_negation: whether we should apply NLTK's mark_negation to document before applying the feature function
    :returns: dict with entries 'contains(f)=1/0' for binary features or 'count(f)=n' for count features
    """
    output = defaultdict(float)
    if mark_negation:
        document = util.mark_negation(document)

    for f in feature_set:
        if f in document:
            if binary:
                output[f'contains({f})'] = 1.
            else:
                output[f'contains({f})'] = document.count(f)
    if not output:
        output['EMPTY()'] = 1
        
    return output

In [5]:
def make_cpd(raw_counts, alpha, v):
    """
    This converts a dictionary of raw counts into a cpd.

    :param raw_counts: dict where a key is a feature and a value is its counts (without pseudo counts)
        this should already include the 'EMPTY()' feature
    :param alpha: how many pseudo counts should we add per observation
    :param v: the size of the feature set (already including the 'EMPTY()' feature)
    :returns: a cpd as a dict where a key is a feature and a value is its smoothed probability
    """
    output = dict()
    all_counts = sum(raw_counts.values())
    for key, count in raw_counts.items():
        output[key] = (count + alpha) / (all_counts + (alpha * v))

    return output

class NaiveBayesClassifier:
    
    def __init__(self, training_pos, training_neg, binary, mark_negation, alpha=0.1, min_freq=2):
        """
        :param training_pos: positive documents
            a document is a list of tokens
        :param training_neg: negative documents
            a document is a list of tokens
        :param binary: whether we are using binary or count features
        :param mark_negation: whether we are pre-processing words in negation scope
        :param alpha: Laplace smooth pseudo count
        :param min_freq: minimum frequency of a token for it to be considered a feature
        """
                
        # Make feature set
        print('Extracting features:')
        feature_set = make_unigram_feature_set(
            training_pos + training_neg,  # we use a concatenation of positive and negative training instances
            min_freq=min_freq, 
            mark_negation=mark_negation)
        
        print(' %d features' % len(feature_set))
                
        # Estimate model: 1/2) count        
        print('MLE: counting')        
        counts = [defaultdict(float), defaultdict(float)]
        for docs, y in [(training_pos, 1), (training_neg, 0)]:
            for doc in docs:  # for each document
                # we extract features
                fmap = make_feature_map(doc, 
                                        feature_set, 
                                        binary=binary, 
                                        mark_negation=mark_negation)
                # and gather counts for the pair (y, f)
                for f, n in fmap.items():
                    counts[y][f] += n  
                                
        # 2/2) Laplace-1 MLE
        #  we put EMPTY() is in the support
        print('MLE: smoothing')
        counts[0]['EMPTY()'] += 0
        counts[1]['EMPTY()'] += 0
        # and compute cpds using Laplace smoothing
        self._cpds = [
            make_cpd(counts[0], alpha, len(feature_set) + 1),  # we add 1 because we want EMPTY() to add towards total
            make_cpd(counts[1], alpha, len(feature_set) + 1)]
        print('MLE: done')
            
        # Store data
        self._binary = binary
        self._mark_negation = mark_negation
        self._alpha = alpha
        self._feature_set = feature_set
            
    def get_log_parameter(self, f, y):
        """Returns log P(f|y)"""
        return np.log(self._cpds[y].get(f, self._cpds[y]['EMPTY()']))
        
    def classify(self, doc):
        """
        This function classifies a document by extracting features <f_1...f_n> for it 
         and then computing 
            log P(<f_1...f_n>|Y=0) and log P(<f_1...f_n>|Y=1)
         and finally picking the best (that is, either Y=0 or Y=1).
        
        :param doc: a list of tokens
        :returns: 0 or 1 (the argmax of log P(<f_1...f_n>|y))
        """
        output = []
        fmap = make_feature_map(doc, self._feature_set, self._binary, self._mark_negation)
        
        log_sum_0 = 0
        log_sum_1 = 0
        for feature in fmap:
            log_sum_0 += self.get_log_parameter(feature, 0)
            log_sum_1 += self.get_log_parameter(feature, 1)
        
        return np.argmax([log_sum_0, log_sum_1])

In [6]:
def evaluate_model(classifier, pos_docs, neg_docs):
    """
    :param classifier: an NaiveBayesClassifier object
    :param pos_docs: positive documents
    :param neg_docs: negative documents
    :returns: a dictionary containing the number of
        * true positives
        * true negatives
        * false positives
        * false negatives
     as well as 
        * accuracy
        * precision
        * recall 
        * and [F1](https://en.wikipedia.org/wiki/F1_score)
    """
    output = {'TP' : 0, 'TN' : 0, "FP" : 0, "FN" : 0}
    for pos_doc in pos_docs:
        if classifier.classify(pos_doc) == 1:
            output['TP'] += 1
        else:
            output['FN'] += 1
        
    for neg_doc in neg_docs:
        if classifier.classify(neg_doc) == 1:
            output['FP'] += 1
        else:
            output['TN'] += 1

    output['A'] = (output['TP'] + output['TN']) / (output['TP'] + output['TN'] + output['FP'] + output['FN'])
    output['P'] = output['TP'] / (output['TP'] + output['FP'])
    output['R'] = output['TP'] / (output['TP'] + output['FN'])
    output['F1'] = 2 * ((output['P'] * output['R']) / (output['P'] + output['R']))
    
    
    return output

In [7]:
def extend_dataframe(filename):
    data = pd.read_csv(filename)
    data.tokens = data.tokens.apply(literal_eval)
    data.lemmed = data.lemmed.apply(literal_eval)
    data.stemmed = data.stemmed.apply(literal_eval)
    
    data["label"] = data["label"].astype(bool)
    data["pos"] = data["label"]
    data["neg"] = ~data["label"]
    
    return data

# Implementation

In [30]:
#------------HYPERPARAMETERS-----------#
# Parameter for input data
INPUT = "stemmed" # "stemmed" / "tokens" / "lemmed"

# Parameters for creating the classifier
BINARY = True
MARK_NEG = False
ALPHA = 1.
MIN_FREQ = 2
#--------------------------------------#

In [31]:
train_data = extend_dataframe("cleaned_data/cleaned_data_train_with_lem_stem.csv")

In [32]:
training_pos = list(train_data[train_data["pos"]][INPUT])
training_neg = list(train_data[train_data["neg"]][INPUT])

In [33]:
classifier1 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=BINARY, mark_negation=MARK_NEG,
    alpha=ALPHA, min_freq=MIN_FREQ)

Extracting features:
 40428 features
MLE: counting
MLE: smoothing
MLE: done


In [34]:
test_data = extend_dataframe("cleaned_data/cleaned_data_test_with_lem_stem.csv")

In [35]:
test_pos = list(test_data[test_data["pos"]][INPUT])
test_neg = list(test_data[test_data["neg"]][INPUT])

In [36]:
evaluate_model(classifier1, test_pos, test_neg)

{'TP': 9663,
 'TN': 11021,
 'FP': 1479,
 'FN': 2837,
 'A': 0.82736,
 'P': 0.8672590199246096,
 'R': 0.77304,
 'F1': 0.8174435326960493}